## Import Libraries and Install Packages

In [ ]:
!pip install seaborn

In [ ]:
%matplotlib inline

import os
import h5py
import random
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold

## Load Data

In [ ]:
IMG_DIR = "/tf/notebooks/hezo/stroke_zurich/data/" 
# IMG_DIR2 = "/tf/notebooks/kook/data-sets/stroke-lh/"
OUTPUT_DIR = "/tf/notebooks/brdd/xAI_3d/data/"

In [ ]:
path_img = IMG_DIR + 'dicom_2d_192x192x3_clean_interpolated_18_02_2021_preprocessed2.h5'
path_tab = IMG_DIR + 'baseline_data_zurich_prepared.csv'

with h5py.File(path_img, "r") as h5:
# with h5py.File(IMG_DIR2 + 'dicom-3d.h5', "r") as h5:
# both images are the same
    X_in = h5["X"][:]
    Y_img = h5["Y_img"][:]
    Y_pat = h5["Y_pat"][:]
    pat = h5["pat"][:]

X_in = np.expand_dims(X_in, axis = 4)

print("image shape in: ", X_in.shape)
print("image min, max, mean, std: ", X_in.min(), X_in.max(), X_in.mean(), X_in.std())


## read tabular data
dat = pd.read_csv(path_tab, sep=",")

print("tabular shape in: ", dat.shape)

In [ ]:
n = []
for p in pat:
    if p in dat.p_id.values:
        n.append(p)
n = len(n)

# match image and tabular data
X = np.zeros((n, X_in.shape[1], X_in.shape[2], X_in.shape[3], X_in.shape[4]))
X_tab = np.zeros((n, 13))
Y_mrs = np.zeros((n))
Y_eventtia = np.zeros((n))
p_id = np.zeros((n))

i = 0
for j, p in enumerate(pat):
    if p in dat.p_id.values:
        k = np.where(dat.p_id.values == p)[0]
        X_tab[i,:] = dat.loc[k,["age", "sexm", "nihss_baseline", "mrs_before",
                               "stroke_beforey", "tia_beforey", "ich_beforey", 
                               "rf_hypertoniay", "rf_diabetesy", "rf_hypercholesterolemiay", 
                               "rf_smokery", "rf_atrial_fibrillationy", "rf_chdy"]]
        X[i] = X_in[j]
        p_id[i] = pat[j]
        Y_eventtia[i] = Y_pat[j]
        Y_mrs[i] = dat.loc[k, "mrs3"]
        i += 1
p_id = p_id.astype("int")
        
print("X img out shape: ", X.shape)
print("X tab out shape: ", X_tab.shape)
print("Y mrs out shape: ", Y_mrs.shape)

In [ ]:
## all mrs <= 2 are favorable all higher unfavorable
Y_new = []
for element in Y_mrs:
    if element in [0,1,2]:
        Y_new.append(0)
    else:
        Y_new.append(1)
Y_new = np.array(Y_new)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)
sns.countplot(x = Y_mrs, ax = ax1)
sns.countplot(x = Y_new, ax = ax2)

In [ ]:
# Safe ids in pd
id_tab = pd.DataFrame(
    {"p_idx": range(1, len(p_id)+1),
     "p_id": p_id,
     "mrs": Y_mrs,
     "unfavorable": Y_new
    }
)

## Define Splits

In [ ]:
# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=999)
# 10 Fold sigmoid random_state 100
# 10 Fold softmax random_state 999
folds = []
  

# for train_index, test_index in skf.split(p_id, Y_new): # 10 Fold sigmoid stratified with Outcome Good/Bad
for train_index, test_index in skf.split(p_id, Y_mrs): # 10 Fold sigmoid stratified with Outcome MRS
    folds.append(p_id[test_index])
    print(sum(Y_new[test_index]))

In [ ]:
for fold in folds:
    print(len(fold))

In [ ]:
for i, fold in enumerate(folds):
    id_tab["fold" + str(i)] = "train" 
    
    # increment for val (+5 so that not twice has only 40 in train & test)
    j = i+5
    if j >= len(folds):
        j = j-10

    id_tab.loc[id_tab["p_id"].isin(fold), "fold"+str(i)] = "test"
    id_tab.loc[id_tab["p_id"].isin(folds[j]), "fold"+str(i)] = "val"

In [ ]:
id_tab

In [ ]:
for i in range(len(folds)):
    print(id_tab["fold"+str(i)].value_counts())

## Save Data

In [ ]:
# id_tab.to_csv(OUTPUT_DIR + "10Fold_sigmoid_ids.csv",  index=False)
id_tab.to_csv(OUTPUT_DIR + "10Fold_softmax_ids_V1.csv",  index=False)

In [ ]:
X = X.squeeze()
X = np.float32(X)

np.save(OUTPUT_DIR + "prepocessed_dicom_3d.npy", X)

## Analyze Data

In [ ]:
# id_tab = pd.read_csv(OUTPUT_DIR + "10Fold_ids_V0.csv", sep=",")
id_tab = pd.read_csv(OUTPUT_DIR + "10Fold_ids_V1.csv", sep=",")
X = np.load(OUTPUT_DIR + "prepocessed_dicom_3d.npy")

In [ ]:
id_tab["unfavorable"].value_counts()

In [ ]:
for i in range(10):
    fig, (ax1, ax2) = plt.subplots(1,2)
    sns.countplot(x = id_tab[id_tab["fold"+str(i)]=="test"].mrs, ax = ax1)
    sns.countplot(x = id_tab[id_tab["fold"+str(i)]=="test"].unfavorable, ax = ax2)

### Check Images

Check if images are the same when accessing them.

In [ ]:
patient = 460
index1 = id_tab[id_tab.p_id == patient].index
print(index1)
index2 = np.argwhere(pat == patient).squeeze()
print(index2)

In [ ]:
im1 = X[index1].astype("float64")
im2 = X_in.squeeze()[index2].astype("float64")
np.allclose(im1, im2)